In [1]:
import numpy as np
from math import sqrt
from sklearn import datasets, linear_model, metrics
from pprint import pprint
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_predict, cross_val_score, cross_validate, train_test_split, KFold
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectPercentile, f_regression
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Carga de datos.
dataset = datasets.fetch_california_housing()
X = dataset.data
y = dataset.target

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle= True, test_size=0.2, random_state=42)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

standardizer = preprocessing.StandardScaler()
X_train = standardizer.fit_transform(X_train)
X_test = standardizer.transform(X_test)

(16512, 8)
(4128, 8)
(16512,)
(4128,)


# KNN

 class sklearn.neighbors.KNeighborsRegressor(n_neighbors=5, *, weights='uniform', algorithm='auto', leaf_size=30, p=2, metric='minkowski', metric_params=None, n_jobs=None)
 
 

- n_neighbors = int, default=5

    Number of neighbors to use by default for kneighbors queries.
    
- weights = {'uniform', 'distance'}, callable or None, default=’uniform’

    Weight function used in prediction. Possible values:

    'uniform' : uniform weights. All points in each neighborhood are weighted equally.

    'distance' : weight points by the inverse of their distance. in this case, closer neighbors of a query point will have      a greater influence than neighbors which are further away.
    [callable] : a user-defined function which accepts an array of distances, and returns an array of the same shape           containing the weights.Uniform weights are used by default.
    
    
- algorithm{'auto', 'ball_tree', 'kd_tree', 'brute'}, default=’auto’

     Algorithm used to compute the nearest neighbors:

     'ball_tree' will use BallTree

     'kd_tree' will use KDTree

     'brute' will use a brute-force search.

     'auto' will attempt to decide the most appropriate algorithm based on the values passed to fit method.
     Note: fitting on sparse input will override the setting of this parameter, using brute force.
     
- metric = str or callable, default=’minkowski’

    Metric to use for distance computation. Default is “minkowski”, which results in the standard Euclidean distance when p     = 2. See the documentation of scipy.spatial.distance and the metrics listed in distance_metrics for valid metric           values.
    
    If metric is “precomputed”, X is assumed to be a distance matrix and must be square during fit. X may be a sparse           graph, in which case only “nonzero” elements may be considered neighbors.

    If metric is a callable function, it takes two arrays representing 1D vectors as inputs and must return one value           indicating the distance between those vectors. This works for Scipy’s metrics, but is less efficient than passing the       metric name as a string.
    
'cityblock' metrics.pairwise.manhattan_distances 

'cosine' metrics.pairwise.cosine_distances 

'euclidean' metrics.pairwise.euclidean_distance 

'haversine'metrics.pairwise.haversine_distances 

'l1' metrics.pairwise.manhattan_distances 

'l2' metrics.pairwise.euclidean_distances 

'manhattan' metrics.pairwise.manhattan_distances 

'nan_euclidean' metrics.pairwise.nan_euclidean_distances


In [3]:
from sklearn.model_selection import GridSearchCV

In [4]:
knn = KNeighborsRegressor()

h_params = {'n_neighbors': list(range(5, 17, 2)),
            'weights': ['uniform', 'distance'],
            'metric': ['euclidean', 'cosine','manhattan']}

GD_KNN = GridSearchCV(estimator=knn,
                          param_grid=h_params,
                          scoring = 'r2',
                          cv= KFold(n_splits=5, shuffle=True, random_state=42),
                          n_jobs= -1)

GD_KNN.fit(X_train, y_train)

print(GD_KNN.best_estimator_)
print(GD_KNN.best_score_)
best_knn = GD_KNN.best_estimator_

KNeighborsRegressor(metric='manhattan', n_neighbors=11, weights='distance')
0.7285053450165693


In [6]:
metricas = {
  'MAE' : lambda y, y_pred: metrics.mean_absolute_error(y, y_pred),
  'RMSE': lambda y, y_pred: sqrt(metrics.mean_squared_error(y, y_pred)),
  'MAPE': lambda y, y_pred: np.mean(np.abs((y - y_pred) / y)) * 100,
  'R2'  : lambda y, y_pred: metrics.r2_score(y, y_pred)}

y_pred = best_knn.predict(X_test)

for nombre, metrica in metricas.items():
    print(f'para la metrica \t {nombre} \t  se obtienen un valor de \t  ---> \t {metrica(y_test, y_pred)}')

para la metrica 	 MAE 	  se obtienen un valor de 	  ---> 	 0.4067905667958895
para la metrica 	 RMSE 	  se obtienen un valor de 	  ---> 	 0.6050943939216056
para la metrica 	 MAPE 	  se obtienen un valor de 	  ---> 	 22.96111407023188
para la metrica 	 R2 	  se obtienen un valor de 	  ---> 	 0.7205916384617095
